# Question 1

## Question 1.1

In [1]:
# symbolic computation tools
import sympy as sp
from sympy import symbols, pprint
from sympy import sin, cos, asin, acos, pi
from sympy import Matrix, simplify, Function, diff, Derivative, nsimplify

from matplotlib.lines import Line2D

import numpy as np

# from IPython import display # for the animation
import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
#plt.ion()


# styling for plots
mpl.rcParams['axes.titlesize'] = 24
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['lines.markersize'] = 10
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16

In [2]:
# We wrap in parentheses here so we can write it on multiple lines. Similar
# with the triple quotes on the string. Usually we don't need to use these things.
(t, 
 theta_1, 
 theta_2, 
 theta_3, 
 l_1, 
 l_2, 
 l_3, 
 s) = symbols("""t, 
                 theta_1 
                 theta_2 
                 theta_3 
                 l_1 
                 l_2 
                 l_3
                 s""" , real = True)


theta_1 = Function('theta_1', real=True)(t)
theta_2 = Function('theta_2', real=True)(t)
theta_3 = Function('theta_3', real=True)(t)

In [3]:
def T(theta, x, y):
    """
    Function to return an arbitrary transformation matrix 
    This is for sympy symbolic calculation
    """
    return Matrix([[cos(theta), -sin(theta), x], 
                   [sin(theta), cos(theta), y],
                   [0, 0, 1]])

def sym_to_np(T):
    return np.array(T).astype(np.float64)

In [4]:
def draw_robot(ordered_list_of_transformations):
    """
    Draw the configuration of the three-link jumper.
    """
    ax_lw = 3
    link_lw = 1

    # homogeneous unit vectors 
    x = np.array([[1, 0, 1]]).T 
    y = np.array([[0, 1, 1]]).T
    origin = np.array([[0, 0, 1]]).T

    prev_origin = np.array([[0, 0, 1]]).T    
    current_transformation = np.eye(3)

    plt.clf()
    ax = plt.gca()  # get current axis

    # now we plot
    plt.plot([prev_origin[0][0]], [prev_origin[1][0]], '-ko',linewidth=link_lw)
    plt.plot([prev_origin[0][0], x[0][0]], [prev_origin[1][0], x[1][0]], '-ro', linewidth=ax_lw)
    plt.plot([prev_origin[0][0], y[0][0]], [prev_origin[1][0], y[1][0]], '-go', linewidth=ax_lw)


    # loop the transforms in order of T_01, T_12, ... , T_N-1N
    for k, transform in enumerate(ordered_list_of_transformations):

        # update the transformation
        current_transformation = current_transformation @ transform
        new_origin = current_transformation @ origin
        new_x = current_transformation @ x
        new_y = current_transformation @ y

        # now we plot
        plt.plot([prev_origin[0][0], new_origin[0][0]], 
                 [prev_origin[1][0], new_origin[1][0]], 
                 '-ko',linewidth=link_lw)
        
        plt.plot([new_origin[0][0], new_x[0][0]], [new_origin[1][0], new_x[1][0]], '-r', linewidth=ax_lw)
        plt.plot([new_origin[0][0], new_y[0][0]], [new_origin[1][0], new_y[1][0]], '-g', linewidth=ax_lw)


#         ax.annotate('{'+str(k+1)+'}', xy=new_origin[0:2][0], xytext=new_origin[0:2][0])

        # lastly update the current_origin to the new_origin
        prev_origin = new_origin

    # lastly set the axes to be equal and appropriate
    ax.set_aspect('equal')
    ax.set_xlabel('x')
    ax.set_ylabel('y')    

In [5]:
T_01 = T(theta_1, 0, 0)
T_12 = T(theta_2, l_1, 0)
T_23 = T(theta_3, l_2, 0)


In [6]:
theta1 = 0 * np.pi/180 
theta2 = 0 * np.pi/180
theta3 = 0 * np.pi/180
link1 = 1
link2 = 1
link3 = 1

# T_01_real = T_np(theta0, 0, 0)
# T_12_real = T_np(theta1, link1, 0)
# T_23_real = T_np(theta2, link2, 0)
# T_34_real = T_np(0, link3, 0)

# # 'subs' substitutes in the numeric value for the variable, evalf evaluates the expression 
T_01_real = sym_to_np(T_01.subs([(theta_1, theta1)]))
T_12_real = sym_to_np(T_12.subs([(theta_2, theta2), (l_1, link1)]))
T_23_real = sym_to_np(T_23.subs([(theta_3, theta3), (l_2, link2)]))
T_34_real = sym_to_np(T(0, link3, 0))

plt.figure(figsize = (8,8))
draw_robot([T_01_real, T_12_real, T_23_real, T_34_real])

# sc = 25
# plt.axis([-sc,sc, -sc, sc])


In [7]:
FK1 = (T_01 * Matrix([[s], [0], [1]]))[:-1,0]
FK1

Matrix([
[s*cos(theta_1(t))],
[s*sin(theta_1(t))]])

In [8]:
FK2 = simplify((T_01 * T_12 * Matrix([[s], [0], [1]]))[:-1,0])
FK2

Matrix([
[l_1*cos(theta_1(t)) + s*cos(theta_1(t) + theta_2(t))],
[l_1*sin(theta_1(t)) + s*sin(theta_1(t) + theta_2(t))]])

In [9]:
FK3 = simplify((T_01 * T_12 * T_23 * Matrix([[s], [0], [1]]))[:-1,0])
FK3

Matrix([
[l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t)) + s*cos(theta_1(t) + theta_2(t) + theta_3(t))],
[l_1*sin(theta_1(t)) + l_2*sin(theta_1(t) + theta_2(t)) + s*sin(theta_1(t) + theta_2(t) + theta_3(t))]])

In [10]:
J1 = FK1.jacobian([theta_1, theta_2, theta_3])
J1

Matrix([
[-s*sin(theta_1(t)), 0, 0],
[ s*cos(theta_1(t)), 0, 0]])

In [11]:
J2 = simplify(FK2.jacobian([theta_1, theta_2, theta_3]))
J2

Matrix([
[-l_1*sin(theta_1(t)) - s*sin(theta_1(t) + theta_2(t)), -s*sin(theta_1(t) + theta_2(t)), 0],
[ l_1*cos(theta_1(t)) + s*cos(theta_1(t) + theta_2(t)),  s*cos(theta_1(t) + theta_2(t)), 0]])

In [12]:
J3 = simplify(FK3.jacobian([theta_1, theta_2, theta_3]))
J3

Matrix([
[-l_1*sin(theta_1(t)) - l_2*sin(theta_1(t) + theta_2(t)) - s*sin(theta_1(t) + theta_2(t) + theta_3(t)), -l_2*sin(theta_1(t) + theta_2(t)) - s*sin(theta_1(t) + theta_2(t) + theta_3(t)), -s*sin(theta_1(t) + theta_2(t) + theta_3(t))],
[ l_1*cos(theta_1(t)) + l_2*cos(theta_1(t) + theta_2(t)) + s*cos(theta_1(t) + theta_2(t) + theta_3(t)),  l_2*cos(theta_1(t) + theta_2(t)) + s*cos(theta_1(t) + theta_2(t) + theta_3(t)),  s*cos(theta_1(t) + theta_2(t) + theta_3(t))]])

## Question 1.2

In [13]:
# final joint vals to arrive at
theta1 = 90 * np.pi/180 
theta2 = 60 * np.pi/180
theta3 = 30 * np.pi/180

time = np.linspace(0, 4 * theta1, 50)

theta1_time = np.pi / 2 * np.sin(time)
theta2_time = - np.pi / 3 * np.sin(2 * time)
theta3_time = - np.pi / 6 * np.sin(4 * time)

dtheta1_time = np.gradient(theta1_time)
dtheta2_time = np.gradient(theta2_time)
dtheta3_time = np.gradient(theta3_time)

fig1 = plt.figure(1,figsize = (8,8))

In [14]:
for theta1_current, theta2_current, theta3_current, v1, v2, v3 in zip(theta1_time, theta2_time, theta3_time, dtheta1_time, dtheta2_time, dtheta3_time):
    # make t matrices
    T_01_real = sym_to_np(T_01.subs([(theta_1, theta1_current)]))
    T_12_real = sym_to_np(T_12.subs([(theta_2, theta2_current), (l_1, link1)]))
    T_23_real = sym_to_np(T_23.subs([(theta_3, theta3_current), (l_2, link2)]))
    T_34_real = sym_to_np(T(0, link3, 0))
    
    draw_robot([T_01_real, T_12_real, T_23_real, T_34_real])
    
    for ss in np.linspace(0.1, 1, 10):
        
        vel1 = J1.subs([(s, ss*link1), 
                       (l_1, link1), 
                       (l_2, link2), 
                       (l_3, link3), 
                       (theta_1, theta1_current),
                       (theta_2, theta2_current),
                       (theta_3, theta3_current)])*Matrix([[v1], [v2], [v3]])
        pos1 = FK1.subs([(s, ss*link1), 
                       (l_1, link1), 
                       (l_2, link2), 
                       (l_3, link3), 
                       (theta_1, theta1_current),
                       (theta_2, theta2_current),
                       (theta_3, theta3_current)])

        # print(pos1)
        plt.quiver(float(pos1[0]), float(pos1[1]), float(vel1[0]), float(vel1[1]))
        
        vel2 = J2.subs([(s, ss*link2), 
                       (l_1, link1), 
                       (l_2, link2), 
                       (l_3, link3), 
                       (theta_1, theta1_current),
                       (theta_2, theta2_current),
                       (theta_3, theta3_current)])*Matrix([[v1], [v2], [v3]])
        pos2 = FK2.subs([(s, ss*link2), 
                       (l_1, link1), 
                       (l_2, link2), 
                       (l_3, link3), 
                       (theta_1, theta1_current),
                       (theta_2, theta2_current),
                       (theta_3, theta3_current)])
        
        # print(pos1)
        plt.quiver(float(pos2[0]), float(pos2[1]), float(vel2[0]), float(vel2[1]))
        
        vel3 = J3.subs([(s, ss*link3), 
               (l_1, link1), 
               (l_2, link2), 
               (l_3, link3), 
               (theta_1, theta1_current),
               (theta_2, theta2_current),
               (theta_3, theta3_current)])*Matrix([[v1], [v2], [v3]])
        pos3 = FK3.subs([(s, ss*link3), 
                       (l_1, link1), 
                       (l_2, link2), 
                       (l_3, link3), 
                       (theta_1, theta1_current),
                       (theta_2, theta2_current),
                       (theta_3, theta3_current)])
        
        # print(pos1)
        plt.quiver(float(pos3[0]), float(pos3[1]), float(vel3[0]), float(vel3[1]))
    
    sc = 25
    plt.axis([-5, 5, -5, 5])
    plt.draw()
    plt.pause(0.01)

## Question 1.3

In [15]:
fig2 = plt.figure(2,figsize = (8,8))

for theta1_current, theta2_current, theta3_current, v1, v2, v3 in zip(theta1_time, theta2_time, theta3_time, dtheta1_time, dtheta2_time, dtheta3_time):
    # make t matrices
    T_01_real = sym_to_np(T_01.subs([(theta_1, theta1_current)]))
    T_12_real = sym_to_np(T_12.subs([(theta_2, theta2_current), (l_1, link1)]))
    T_23_real = sym_to_np(T_23.subs([(theta_3, theta3_current), (l_2, link2)]))
    T_34_real = sym_to_np(T(0, link3, 0))
    
#    draw_robot([T_01_real, T_12_real, T_23_real, T_34_real])
    
    viscous_forces_x = 0
    viscous_forces_y = 0
    
    for ss in np.linspace(0.1, 1, 10):
        
        vel1 = J1.subs([(s, ss*link1), 
                       (l_1, link1), 
                       (l_2, link2), 
                       (l_3, link3), 
                       (theta_1, theta1_current),
                       (theta_2, theta2_current),
                       (theta_3, theta3_current)])*Matrix([[v1], [v2], [v3]])
        pos1 = FK1.subs([(s, ss*link1), 
                       (l_1, link1), 
                       (l_2, link2), 
                       (l_3, link3), 
                       (theta_1, theta1_current),
                       (theta_2, theta2_current),
                       (theta_3, theta3_current)])
        
        # print(pos1)
        #plt.quiver(float(pos1[0]), float(pos1[1]), float(vel1[0]), float(vel1[1]))
        
        vel2 = J2.subs([(s, ss*link2), 
                       (l_1, link1), 
                       (l_2, link2), 
                       (l_3, link3), 
                       (theta_1, theta1_current),
                       (theta_2, theta2_current),
                       (theta_3, theta3_current)])*Matrix([[v1], [v2], [v3]])
        pos2 = FK2.subs([(s, ss*link2), 
                       (l_1, link1), 
                       (l_2, link2), 
                       (l_3, link3), 
                       (theta_1, theta1_current),
                       (theta_2, theta2_current),
                       (theta_3, theta3_current)])
        
        # print(pos1)
        #plt.quiver(float(pos2[0]), float(pos2[1]), float(vel2[0]), float(vel2[1]))
        
        vel3 = J3.subs([(s, ss*link3), 
               (l_1, link1), 
               (l_2, link2), 
               (l_3, link3), 
               (theta_1, theta1_current),
               (theta_2, theta2_current),
               (theta_3, theta3_current)])*Matrix([[v1], [v2], [v3]])
        pos3 = FK3.subs([(s, ss*link3), 
                       (l_1, link1), 
                       (l_2, link2), 
                       (l_3, link3), 
                       (theta_1, theta1_current),
                       (theta_2, theta2_current),
                       (theta_3, theta3_current)])
        
        # print(pos1)
        #plt.quiver(float(pos3[0]), float(pos3[1]), float(vel3[0]), float(vel3[1]))
        
        viscous_forces_x =  viscous_forces_x - (vel1[0] + vel2[0] + vel3[0])
        viscous_forces_y =  viscous_forces_y - (vel1[1] + vel2[1] + vel3[1])
        
    plt.plot(viscous_forces_x, viscous_forces_y, 'ro')
#    print(viscous_forces_y)
    
    sc = 25
    plt.axis([-20, 20, -20, 20])
#    plt.draw()
    plt.pause(0.01)

## Question 1.4

In [16]:
fig3 = plt.figure(3,figsize = (8,8))

count = 0

for theta1_current, theta2_current, theta3_current, v1, v2, v3 in zip(theta1_time, theta2_time, theta3_time, dtheta1_time, dtheta2_time, dtheta3_time):
    # make t matrices
    T_01_real = sym_to_np(T_01.subs([(theta_1, theta1_current)]))
    T_12_real = sym_to_np(T_12.subs([(theta_2, theta2_current), (l_1, link1)]))
    T_23_real = sym_to_np(T_23.subs([(theta_3, theta3_current), (l_2, link2)]))
    T_34_real = sym_to_np(T(0, link3, 0))
    
#    draw_robot([T_01_real, T_12_real, T_23_real, T_34_real])
    
    viscous_forces_x = 0
    viscous_forces_y = 0
    
    for ss in np.linspace(0.1, 1, 10):
        
        vel1 = J1.subs([(s, ss*link1), 
                       (l_1, link1), 
                       (l_2, link2), 
                       (l_3, link3), 
                       (theta_1, theta1_current),
                       (theta_2, theta2_current),
                       (theta_3, theta3_current)])*Matrix([[v1], [v2], [v3]])
        pos1 = FK1.subs([(s, ss*link1), 
                       (l_1, link1), 
                       (l_2, link2), 
                       (l_3, link3), 
                       (theta_1, theta1_current),
                       (theta_2, theta2_current),
                       (theta_3, theta3_current)])
        
        # print(pos1)
        #plt.quiver(float(pos1[0]), float(pos1[1]), float(vel1[0]), float(vel1[1]))
        
        vel2 = J2.subs([(s, ss*link2), 
                       (l_1, link1), 
                       (l_2, link2), 
                       (l_3, link3), 
                       (theta_1, theta1_current),
                       (theta_2, theta2_current),
                       (theta_3, theta3_current)])*Matrix([[v1], [v2], [v3]])
        pos2 = FK2.subs([(s, ss*link2), 
                       (l_1, link1), 
                       (l_2, link2), 
                       (l_3, link3), 
                       (theta_1, theta1_current),
                       (theta_2, theta2_current),
                       (theta_3, theta3_current)])
        
        # print(pos1)
        #plt.quiver(float(pos2[0]), float(pos2[1]), float(vel2[0]), float(vel2[1]))
        
        vel3 = J3.subs([(s, ss*link3), 
               (l_1, link1), 
               (l_2, link2), 
               (l_3, link3), 
               (theta_1, theta1_current),
               (theta_2, theta2_current),
               (theta_3, theta3_current)])*Matrix([[v1], [v2], [v3]])
        pos3 = FK3.subs([(s, ss*link3), 
                       (l_1, link1), 
                       (l_2, link2), 
                       (l_3, link3), 
                       (theta_1, theta1_current),
                       (theta_2, theta2_current),
                       (theta_3, theta3_current)])
        
        # print(pos1)
        #plt.quiver(float(pos3[0]), float(pos3[1]), float(vel3[0]), float(vel3[1]))
        
        viscous_forces_x =  viscous_forces_x - (vel1[0] + vel2[0] + vel3[0])
        viscous_forces_y =  viscous_forces_y - (vel1[1] + vel2[1] + vel3[1])
        
    torques = J3.T.subs([(s, 1), 
                       (l_1, link1), 
                       (l_2, link2), 
                       (l_3, link3), 
                       (theta_1, theta1_current),
                       (theta_2, theta2_current),
                       (theta_3, theta3_current)]) @ Matrix([viscous_forces_x, viscous_forces_y])
    
#    print(torques)
    
#    print(time[count])
    plt.plot(time[count], torques[0], 'ro')
    plt.plot(time[count], torques[1], 'bo')
    plt.plot(time[count], torques[2], 'go')
        
#    plt.plot(viscous_forces_x, viscous_forces_y, 'ro')
#    print(viscous_forces_y)
    
    sc = 25
    plt.axis([0, 4 * theta1, -50, 50])
#    plt.draw()
    plt.pause(0.01)
    
    count += 1
    
legend_elements = [Line2D([0], [0], marker='o', color='w', label='tau1',
                          markerfacecolor='r', markersize=15),
                   Line2D([0], [0], marker='o', color='w', label='tau2',
                          markerfacecolor='b', markersize=15),
                   Line2D([0], [0], marker='o', color='w', label='tau3',
                          markerfacecolor='g', markersize=15)]

    
fig3.legend(handles = legend_elements, loc="lower right")

## Question 1.5

In [17]:
fig4 = plt.figure(4,figsize = (8,8))
fig5 = plt.figure(5,figsize = (8,8))
plt.title("Viscous Force vs. Time")
fig6 = plt.figure(5,figsize = (8,8))
plt.title("Net Force vs. Time")

count = 0

net_viscous_forces_x = 0
net_viscous_forces_y = 0

for theta1_current, theta2_current, theta3_current, v1, v2, v3 in zip(theta1_time, theta2_time, theta3_time, dtheta1_time, dtheta2_time, dtheta3_time):
    # make t matrices
    T_01_real = sym_to_np(T_01.subs([(theta_1, theta1_current)]))
    T_12_real = sym_to_np(T(0, link1, 0))
#    T_12_real = sym_to_np(T_12.subs([(theta_2, theta2_current), (l_1, link1)]))
#    T_23_real = sym_to_np(T_23.subs([(theta_3, theta3_current), (l_2, link2)]))
#    T_34_real = sym_to_np(T(0, link3, 0))
    
    draw_robot([T_01_real])
    
    viscous_forces_x = 0
    viscous_forces_y = 0
    
    for ss in np.linspace(0.1, 1, 10):
        
        vel1 = J1.subs([(s, ss*link1), 
                       (l_1, link1), 
                       (l_2, 0), 
                       (l_3, 0), 
                       (theta_1, theta1_current),
                       (theta_2, 0),
                       (theta_3, 0)])*Matrix([[v1], [0], [0]])
        pos1 = FK1.subs([(s, ss*link1), 
                       (l_1, link1), 
                       (l_2, 0), 
                       (l_3, 0), 
                       (theta_1, theta1_current),
                       (theta_2, 0),
                       (theta_3, 0)])

        # print(pos1)
        plt.figure(4)
        plt.quiver(float(pos1[0]), float(pos1[1]), float(vel1[0]), float(vel1[1]))

        viscous_forces_x =  viscous_forces_x - vel1[0]
        viscous_forces_y =  viscous_forces_y - vel1[1]
        net_viscous_forces_x =  net_viscous_forces_x - vel1[0]
        net_viscous_forces_y =  net_viscous_forces_y - vel1[1]
#        print(net_viscous_forces_x)
#        print(net_viscous_forces_y)
        
    plt.figure(5)
    plt.plot(time[count], viscous_forces_x, 'ro')
    plt.plot(time[count], viscous_forces_y, 'bo')
    plt.axis([-0, 8, -4, 4])
    plt.draw()
    
    plt.figure(6)
    plt.plot(time[count], net_viscous_forces_x, 'ro')
    plt.plot(time[count], net_viscous_forces_y, 'bo')
    plt.axis([0, 12, -6, 6])
    plt.draw()
    
    plt.figure(4)
    sc = 25
    plt.axis([-2, 2, -2, 2])
    plt.draw()
    plt.pause(0.01)
    
    count += 1

As seen from the net force graph, the net force is zero when the link is at its initial position.